In [1]:
import numpy as np 
import pandas as pd
import torch

E:\conda_envs\transformers_lm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DBpedia

In [4]:
data_dir ="../all_data/dbpedia_csv/train.csv"

df = pd.read_csv(f"{data_dir}", header= None)

In [5]:
df.head()

,0,1,2
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...


In [16]:
df[0].value_counts()

1     40000
2     40000
3     40000
4     40000
5     40000
6     40000
7     40000
8     40000
9     40000
10    40000
11    40000
12    40000
13    40000
14    40000
Name: 0, dtype: int64

In [12]:
df.shape

(560000, 3)

In [8]:
# load in the preprocessed tar file

# load data
preprocessed_data = torch.load("../all_data/dbpedia_csv/processed/TRAIN_data.pth.tar")

In [14]:
len(preprocessed_data['sents'])

560000

In [21]:
preprocessed_data['labels'][5]

0

In [3]:
import torch
from typing import Tuple, Dict
from collections import Counter
from nltk.tokenize import PunktSentenceTokenizer, TreebankWordTokenizer
from tqdm import tqdm
import pandas as pd
import os
import json


def get_clean_text(text: str) -> str:
    """
    Preprocess text for being used in the model, including lower-casing,
    standardizing newlines and removing junk.

    Parameters
    ----------
    text : str
        A string to be cleaned

    Returns
    -------
    clean_text : str
        String after being cleaned
    """
    if isinstance(text, float):
        return ''

    clean_text = text.lower().replace('<br />', '\n').replace('<br>', '\n').replace('\\n', '\n').replace('&#xd;', '\n')
    return clean_text

# tokenizers
word_tokenizer = TreebankWordTokenizer()

def read_csv(csv_folder: str, split: str, word_limit: int) -> Tuple[list, list, Counter]:
    """
    Read CSVs containing raw training data, clean sentences and labels, and do
    a word-count.

    Parameters
    ----------
    csv_folder : str
        Folder containing the dataset in CSV format files

    split : str
        'train' or 'test' split?

    word_limit : int
        Truncate long sentences to these many words

    Returns
    -------
    sents : list
        Sentences ([ word1, ..., wordn ])

    labels : list
        List of label of each sentence

    word_counter : Counter
    """
    assert split in {'train', 'test'}

    sents = []
    labels = []
    word_counter = Counter()
    data = pd.read_csv(os.path.join(csv_folder, split + '.csv'), header = None)
    for i in tqdm(range(data.shape[0])):
        row = list(data.loc[i, :])

        s = ''

        for text in row[1:]:
            text = get_clean_text(text)
            s = s + text

        words = word_tokenizer.tokenize(s)[:word_limit]
        # if sentence is empty (due to removing punctuation, digits, etc.)
        if len(words) == 0:
            continue
        word_counter.update(words)

        labels.append(int(row[0]) - 1) # since labels are 1-indexed in the CSV
        sents.append(words)

    return sents, labels, word_counter
